In [3]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.drawing.image import Image
import win32com.client as win32
import os
from PIL import ImageGrab
import win32com.client

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [1]:
Today ='2024-04-30'
print_to_excel = False

In [4]:
query = f"""
SELECT *
FROM xxwh."unifiedARReport"
WHERE 
    "rowType" = 'M' and
    "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') 
    """
# "sourceSystem" in ('SAP ECP', 'SAP ERP') and
data_xxwh = export_from_WHWEEK(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:172: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [4]:
# data_xxwh.iloc[:2,150:]
#,partyInternational,customerName,orgName,companyCode,amount,currencyCode,exchangeRate,documentDate,dueDate,presentAmountUSD
data_work = data_xxwh[['companyCode', 'orgName', 'holding', 'functionalCurrency', 'sourceSystem', 'reportDate','documentDate','dueDate','insertToSource','amount','currencyCode','exchangeRate','presentAmountUSD','reportDateUSDRate','debtHoldingCurrency','holdingCurrency','presentationCurrency','reportDateIfrsRateUSDBu','presentAmountIfrsUSD']]
data_work = data_work.fillna(0)
data_USD_1 = data_work[(data_work.currencyCode=='USD')&(data_work.reportDateUSDRate==1)]
data_USD_not1 = data_work[(data_work.currencyCode=='USD')&(data_work.reportDateUSDRate!=1)]
count=0
if print_to_excel == True:
    Output_file=f'{str(date.today())}_USDrate_xxwh-unifiedARReport.xlsx'
    writer = pd.ExcelWriter(Output_file, engine='openpyxl')  
    workbook=writer.book
for USD_data in [data_USD_1, data_USD_not1]:
    count = count+1
    index_list = ['orgName', 'holding',  'reportDate','documentDate','insertToSource','functionalCurrency', 'sourceSystem']
    group_data = USD_data.pivot_table(index=index_list, values=['amount','presentAmountUSD'], aggfunc=sum).reset_index()

    if print_to_excel == True:
        sheet_name=''
        if count == 1:
            sheet_name = 'USD_1'
        if count == 2:
            sheet_name = 'USD_not1'
        group_data.to_excel(writer, sheet_name=sheet_name, index=False)
        col_count=group_data.shape[1]+1
        for col in index_list:
            col_count+=1
            pd.DataFrame({f'Unique_{col}':group_data[col].unique().tolist()}).to_excel(writer, sheet_name=sheet_name, startcol=col_count, index=False)
if print_to_excel == True:
    writer.close()


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12640\3488317984.py:15: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_data = USD_data.pivot_table(index=index_list, values=['amount','presentAmountUSD'], aggfunc=sum).reset_index()
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_12640\3488317984.py:15: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_data = USD_data.pivot_table(index=index_list, values=['amount','presentAmountUSD'], aggfunc=sum).reset_index()


In [40]:
# нужные столбцы для подсчета корреляции
data_check = data_xxwh[['sourceSystem','currencyCode','functionalCurrency','amount','docAmount','exchangeRate','localAmount','presentAmountUSD','reportDateUSDRate']]
# фильтр по функциональной и операционной валюте
data_check_QUERY = data_check[data_check.currencyCode=='BRL'].sort_values(['currencyCode','functionalCurrency'])
# data_check_QUERY = data_check[(data_check.currencyCode!='USD')&(data_check.functionalCurrency!='USD')&(data_check.functionalCurrency!=data_check.currencyCode)].sort_values(['currencyCode','functionalCurrency'])
# фильтр равенства amount и docamount
# data_check_QUERY = data_check_QUERY[data_check_QUERY.amount.abs().astype(int)!=data_check_QUERY.docAmount.abs().astype(int)]
# Выбор валютно пары для подсчетов корреляции
# data_check_QUERY[['currencyCode','functionalCurrency']].value_counts()
# data_check_QUERY = data_check_QUERY[(data_check_QUERY.currencyCode=='CNY')&(data_check_QUERY.functionalCurrency=='RUB')]
# Выбор исключительно столбцов с числами
data_check_QUERY_for_cor = data_check_QUERY.iloc[:,3:]
# Подсчет корреляции
data_check_QUERY_for_cor_cor = data_check_QUERY_for_cor.corr()
data_check_filter_corr = data_check_QUERY_for_cor_cor[(data_check_QUERY_for_cor_cor.abs()>0.87)&(data_check_QUERY_for_cor_cor.abs()!=1.0)]
# Удалить полностью пустые столбцы
data_check_filter_corr.drop(data_check_filter_corr.columns[data_check_filter_corr.isna().all(axis=1)], axis=1, inplace=True)
data_check_filter_corr

,amount,docAmount,localAmount,presentAmountUSD
amount,NaN,NaN,NaN,1.0
docAmount,NaN,NaN,NaN,1.0
exchangeRate,NaN,NaN,NaN,NaN
localAmount,NaN,NaN,NaN,1.0
presentAmountUSD,1.0,1.0,1.0,NaN
reportDateUSDRate,NaN,NaN,NaN,NaN


In [44]:
# data_check_QUERY = add_in_currency_column(data_check_QUERY, 'USD', 'currencyCode', False, 'amount', DATE=Today)
data_check_QUERY['amount_exchangeRate'] = data_check.amount * data_check.exchangeRate
data_check_QUERY['localAmount_reportDateUSDRate'] = data_check.localAmount * data_check.reportDateUSDRate
# data_check_QUERY.to_excel(f'{str(date.today())}_BRL_xxwh-unifiedARReport.xlsx')
data_check_QUERY

,sourceSystem,currencyCode,functionalCurrency,amount,docAmount,exchangeRate,localAmount,presentAmountUSD,reportDateUSDRate,amount_exchangeRate,localAmount_reportDateUSDRate,Coef_to_USD,amount_in_USD
0,ETL document loader,BRL,BRL,-982.52,-982.52,5.1184,-982.52,-191.97,0.195385,-5028.930368,-191.970063,0.192552,-189.186275
1,ETL document loader,BRL,BRL,-982.52,-982.52,5.1184,-982.52,-191.97,0.195385,-5028.930368,-191.970063,0.192552,-189.186275
2,ETL document loader,BRL,BRL,-982.52,-982.52,5.1184,-982.52,-191.97,0.195385,-5028.930368,-191.970063,0.192552,-189.186275
3,ETL document loader,BRL,BRL,-982.52,-982.52,5.1184,-982.52,-191.97,0.195385,-5028.930368,-191.970063,0.192552,-189.186275
4,ETL document loader,BRL,BRL,-982.52,-982.52,5.1184,-982.52,-191.97,0.195385,-5028.930368,-191.970063,0.192552,-189.186275
...,...,...,...,...,...,...,...,...,...,...,...,...,...
618299,SAP_HRG,BRL,BRL,-1347.00,-1347.00,1.0000,-1347.00,-260.45,0.193356,-1347.000000,-260.450000,0.192552,-259.367659
618300,SAP_HRG,BRL,BRL,-1481.70,-1481.70,1.0000,-1481.70,-286.50,0.193359,-1481.700000,-286.500000,0.192552,-285.304425
618301,SAP_HRG,BRL,BRL,-1436.80,-1436.80,1.0000,-1436.80,-277.81,0.193353,-1436.800000,-277.810000,0.192552,-276.658836
618302,SAP_HRG,BRL,BRL,-1347.00,-1347.00,1.0000,-1347.00,-260.45,0.193356,-1347.000000,-260.450000,0.192552,-259.367659


In [26]:
data_xxwh[data_xxwh.currencyCode == 'BRL'].functionalCurrency.unique()

array(['BRL'], dtype=object)

In [7]:
# data_check_QUERY = add_in_currency_column(data_check_QUERY, 'USD', 'currencyCode', False, 'amount', DATE=Today)
data_check = add_in_currency_column(data_check, 'USD', 'currencyCode', False, 'amount', DATE=Today)
data_check[(data_check.reportDateUSDRate/data_check.Coef_to_USD).abs()>1.5]

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CCY_to'] = CCY_to
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = DATE
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

KeyboardInterrupt: 

In [8]:
Output_file = f'{str(date.today())}_huge_differences_xxwh-unifiedARReport.xlsx'
data_USD_not1 = data_xxwh[(data_xxwh.currencyCode=='USD')&(data_xxwh.reportDateUSDRate!=1)]
print_data = data_USD_not1[(((data_USD_not1.amount/data_USD_not1.presentAmountUSD).abs() >2)|((data_USD_not1.presentAmountUSD/data_USD_not1.amount).abs() >2))&(data_USD_not1.amount>1)]
print_data.to_excel(Output_file, sheet_name='USD_not1')
# new_list(data_USD_1, Output_file, 'USD_1')